<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/104_%E3%82%A4%E3%83%B3%E3%83%9D%E3%83%BC%E3%83%88%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

インポートシステム
==================

モジュールとパッケージ
----------------------

### モジュール ###

**モジュール**（module）は、Python の定義や文が実行される環境を表現するものであり、それ自体がオブジェクトである。

モジュールの `__name__` 属性の値は、そのモジュールの名前（文字列）となる。

トップレベルのコードが実行されるモジュールには自動的に `__main__` という名前が付けられる。ここで、トップレベルのコードとは、「Python インタープリター起動時に最初から実行され、プログラムに必要な他のすべてのモジュールをインポートするコード」という意味である。トップレベルのコードは、アプリケーションへの**エントリーポイント**（entry point）と呼ばれることもある。

Python インタープリターの対話モードでトップレベル環境を確認すると、次のようになる。

``` python
>>> __name__
'__main__'
```

`__main__` 以外のモジュールの名前には、インポート時に Python コードが保存されている場所のファイルシステム上の名前が使われる。ファイルなら、拡張子が `.py` でなければならず、そのファイルのベース名がモジュールの名前となる。このとき、ファイルシステムで使用できる文字と、Python 識別子に使用できる文字が異なることに注意する。Python 識別子に使用できない文字がファイルシステム上の名前に含まれる場合、インポート時に構文エラーが発生する。

`__name__` をチェックすれば各モジュールは自分がトップレベル環境で実行されているかどうかを知ることができる。このことから、モジュールがトップレベル環境で実行されている場合でのみ実行されるコードを次のように書ける:

``` python
if __name__ == '__main__':
    # Execute when the module is not initialized from an import statement.
    ...
```

モジュールにスクリプト用途のみのコードが含まれる場合には、そのスクリプト用コードを `if __name__ == '__main__'` の下のブロックに置くとよい。そうすれば、たとえばテスト目的で別のモジュールからインポートされる時に、スクリプト用コードが意図に反して実行されてしまうようなことがない。

`if __name__ == '__main__'` の下のブロックにあるコードはできるだけ少なくした方が、コードの分かりやすさや正確さにつながる。最もよくあるのが、プログラムの主要な処理を `main` 関数の中にカプセル化する方法である:

In [ ]:
def func(a, b):
    print(a + b)
    return a + b

def main():
    """フィボナッチ数列の第2項から第6項までを出力する"""
    x, y = 0, 1
    for i in range(2, 7):
        x, y = y, func(x, y)

if __name__ == "__main__":
    main()

1
2
3
5
8


このコードで、もし `main` 関数内のコードをカプセル化せず `if __name__ == '__main__'` の下に直接書いた場合、 `x`, `y` 変数はモジュール全体からグローバルにアクセスできてしまう。モジュール内の他の関数が意図せずローカル変数ではなくそのグローバル変数を使用してしまう可能性があるため、ミスにつながる。 `main` 関数を用意することでこの問題は解決できる。

### 通常のパッケージ ###

`__path__` 属性を持つモジュールは**パッケージ**（package）と呼ばれる。通常のパッケージでは、`__path__` 属性の値はパッケージが存在するディレクトリへのパスのリストとなる。Python モジュールがファイルシステム上にファイル（`.py` ファイル）として存在するのに対して、Python パッケージはファイルシステム上にディレクトリとして存在する。

ファイルシステム上のディレクトリ階層により、パッケージは階層構造を持つ。上位のパッケージを親パッケージ、下位のパッケージをサブパッケージと呼ぶ。パッケージの名前はドット `.` を含むことで階層構造を表現する。

たとえば、Python 標準ライブラリの `importlib` パッケージには `metadata` サブパッケージがあって、その `__path__` 属性から `importlib` ディレクトリの `metadata` サブディレクトリとしてファイルシステム上に存在することがわかる。また、 `__name__` 属性からサブパッケージの名前は `'importlib.metadata'` とわかる。


In [ ]:
import importlib.metadata
print(f"{importlib.metadata.__path__ = }")
print(f"{importlib.metadata.__name__ = }")

importlib.metadata.__path__ = ['/usr/lib/python3.10/importlib/metadata']
importlib.metadata.__name__ = 'importlib.metadata'


通常のパッケージは、 `__init__.py` ファイルを含むディレクトリとする必要がある。通常のパッケージのインポート時には、この `__init__.py` ファイルが暗黙的に実行される（`__path__` 属性の初期化の後に実行される）。パッケージは単一の `__init__.py` ファイルだけを含むものであってもよい。`__init__.py` はただの空ファイルでも構わない。

たとえば、以下のようなファイルシステム配置は、3 つのサブパッケージを持つ最上位の `parent` パッケージを定義する:

``` text
parent/
    __init__.py
    one/
        __init__.py
        something.py
    two/
        __init__.py
    three/
        __init__.py
```

サブディレクトリ `one`、`two`、`three` が `__init__.py` ファイルを含むので、`parent.one`、`parent.two`、`parent.three` をパッケージとしてインポートすることができる。`parent.one` をインポートすると暗黙的に `parent/__init__.py` と `parent/one/__init__.py` が実行される。その後に `parent.two` もしくは `parent.three` をインポートすると、それぞれ `parent/two/__init__.py` や `parent/three/__init__.py` が実行される。

このパッケージ構成において、各パッケージの名前を確認するために、 `parent/__init__.py` と `parent/one/__init__.py` に以下のようなコードを書く。

`parent/__init__.py`:

``` python
from . import one
name = __name__
one_name = one.name
```

`parent/one/__init__.py`:

``` python
name = __name__
```

ここで、`parent` をサブディレクトリに持つプロジェクトルートで、Python インタープリターを起動すると、次のようにしてパッケージの名前を確認することができる。

``` python
>>> import parent
>>> print(parent.name)
parent
>>> print(parent.one_name)
parent.one
```

### モジュール検索パス ###

標準ライブラリの `sys` モジュールに含まれる `sys.path` 変数は、モジュールを検索するパスを示す文字列のリストである。`PYTHONPATH` 環境変数と、インストール時に指定したデフォルトパスで初期化される。

In [ ]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

`sys.path` に含まれる空文字列 `''` はカレントディレクトリと解釈される。

`sys.path` の最初の要素は、Python インタープリター実行時に指定するインターフェースオプションにより追加される。

``` shell
python [フラグ] [インターフェースオプション] [引数]
```

インターフェースオプションと、`sys.path` の最初の要素との対応は以下の通り:

| オプション | 意味 | `sys.path[0]` |
|:---|:---|:---|
| `-c <command>` | command の Python 文を `__main__` モジュールとして実行する | カレントディレクトリ |
| `-m <module-name>` | 指定されたモジュール名のモジュールを探し、その内容を `__main__` モジュールとして実行する。引数はモジュー<br />ル名なので、拡張子（`.py`）を含めない。パッケージ名を与えた場合、`<pkg>.__main__.py` を `__main__` モジュー<br />ルとして実行する | カレントディレクトリ |
| `<python-file>` | Python ファイル（拡張子 `.py`）を `__main__` モジュールとして実行する | ファイルを含むディレクトリ |
| `<directory>` | ディレクトリの中の `__main__.py` ファイルを `__main__` モジュールとして実行する | `<directory>` |
| `<zip-file>` | zip ファイル（拡張子 `.zip`）の中の `__main__.py` ファイルを `__main__` モジュールとして実行する | `<zip-file>` |
| `-` | 標準入力（`sys.stdin`）から読み込んだ Python コードを `__main__` モジュールとして実行する | カレントディレクトリ |

インターフェースオプションの指定によって安全でない可能性のあるパスが `sys.path` の前に追加されることに注意する。Python インタープリター実行時に `-I`（隔離モード）フラグを指定する場合は、上記のようなパスが `sys.path` の前に追加されなくなる。隔離モードでは、ユーザーがインストールしたパッケージも、全ての `PYTHON*` 環境変数も無視される。

Python 3.11 で `-P` フラグが追加された。Python インタープリター実行時に `-P` フラグを指定する場合は、上記のようなパスが `sys.path` の前に追加されなくなるが、ユーザーがインストールしたパッケージと全ての `PYTHON*` 環境変数は有効である。

`PYTHON*` 環境変数の例:

| 環境変数 | 機能 |
|:---|:---|
| `PYTHONHOME` | Python 標準ライブラリの場所を変更する |
| `PYTHONPATH` | モジュールファイルのデフォルトの検索パスを追加する |
| `PYTHONUSERBASE` | Python ユーザーインストールディレクトリの場所を変更する |
| `PYTHONSTARTUP` | この変数が読み込み可能なファイル名の場合、対話モードで最初のプロンプトが表示される前にそのファイルの Python コマンドが実行される |

### コマンドラインインターフェース ###

パッケージに含まれる `__main__.py` は、コマンドラインインターフェースをサポートするために用いられる。パッケージのコマンドラインインターフェースは、Python インタープリターに渡すインターフェースオプションにて `-m <module-name>` のようにパッケージが指定される形をとる。この場合、暗黙的にパッケージがインポートされた状態で `__main__.py` が実行される。`__init__.py` のインポート時実行がパッケージとしての実行であるのに対して、`__main__.py` の実行は（コマンドラインで `python __main__.py` と実行する場合と同等のトップレベル環境である） `__main__` モジュールとしての実行になる。

たとえば、カレントディレクトの直下に以下のようなファイルシステム配置を行うとする。

``` text
./
    my_package/
        __init__.py
        __main__.py
```

`__init__.py`:

``` python
print("__init__.py: パッケージとして実行された")

if __name__ == "__main__":
    print("__init__.py: __main__ モジュールとして実行された")
```

`__main__.py`:

``` python
print("__main__.py: ここから始まる")

if __name__ == "__main__":
    print("__main__.py: __main__ モジュールとして実行された")
```

コマンド `python -m my_package` を実行する場合、カレントディレクトの直下の `my_package` サブディレクトリは、`sys.path[0]` の値によりモジュール検索パスが通っている状態であることに注意する。実行結果は次のようになる:

``` shell
PS> python -m my_package
__init__.py: パッケージとして実行された
__main__.py: ここから始まる
__main__.py: __main__ モジュールとして実行された
```

`__init__.py` の if 文が実行されていないことに注意する。`__init__.py` は `my_package` パッケージとしてインポート時に実行されるので、`__name__` は `'my_package'` になっているからである。

なお、`-m` を使わないコマンド `python my_package` はインタープリターに直接ディレクトリを指定する形であり、これを実行する場合でも `__main__.py` が `__main__` モジュールとして実行される。ただし、暗黙的な `my_package` のインポートは行われないので、`__init__.py` は実行されない。実行結果は次のようになる:

``` shell
PS> python my_package
__main__.py: ここから始まる
__main__.py: __main__ モジュールとして実行された
```

### 名前空間パッケージ ###

実は、ディレクトリに `__init__.py` が無くてもディレクトリをパッケージとしてインポートすることは可能である。ただし、`__init__.py` が無い場合は、複数のパスに分散する同じ名前のディレクトリを 1 つのパッケージとして扱うようになる。このようなパッケージは通常のパッケージと区別して**名前空間パッケージ**と呼ぶ。

たとえば、以下のようなファイルシステム配置は、2 つのサブパッケージを持つ最上位の `parent` パッケージを定義する:

``` text
./
    spam/
        parent/
            one/
                __init__.py
    ham/
        parent/
            two/
                __init__.py
```

`parent` ディレクトリは `spam` と `ham` の 2 つのディレクトリに存在していて、どちらにも `__init__.py` が存在しないことに注意する。`spam` と `ham` を `sys.path` に含めれば、 `import parent` は名前空間パッケージをインポートすることになる。実際、`__path__` 属性を調べると、`spam` と `ham` のそれぞれの `parent` ディレクトリが認識されていることがわかる。

``` python
>>> import sys
>>> sys.path.append("spam")
>>> sys.path.append("ham")
>>> import parent
>>> parent.__path__
_NamespacePath(['C:\\Users\\User\\Desktop\\tmp\\spam\\parent', 'C:\\Users\\User\\Desktop\\tmp\\ham\\parent'])
```

インポート
----------

### import 文 ###

import 文の構文は次のとおり。

``` python
import <MODULE>[ as <IDENTIFIER>](, <MODULE>[ as <IDENTIFIER>], ...)
from <PACKAGE> import <SUBPACKAGE>[ as <IDENTIFIER>](, <SUBPACKAGE>[ as <IDENTIFIER>], ...)
from <MODULE> import <NAME>[ as <IDENTIFIER>](, <NAME>[ as <IDENTIFIER>], ...)
from <MODULE> import *
```

ここで、 `<MODULE>` はモジュールまたはパッケージの名前（識別子）であり、`<PACKAGE>` と `<SUBPACKAGE>` はそれぞれパッケージとサブパッケージの名前（識別子）である。また、 `<NAME>` は `<MODULE>` に含まれる関数、変数、クラスの名前（識別子）である。

`from <PACKAGE> import` と `from <MODULE> import` では、`import` に続くカンマで区切られた複数の節を明示的に `()` で括れば、次のように改行が可能である:

``` python
from <PACKAGE> import (
    <SUBPACKAGE> as <IDENTIFIER>,
    <SUBPACKAGE> as <IDENTIFIER>,
)
from <MODULE> import (
    <NAME> as <IDENTIFIER>,
    <NAME> as <IDENTIFIER>,
)
```

パッケージをインポートする際、 Python は `sys.path` 上のディレクトリを検索して、トップレベルのパッケージの入ったサブディレクトリを探す。

パッケージ内のモジュールでは、自身が含まれるサブパッケージをドット `.` で参照でき、親パッケージを二重ドット `..` で参照でき、親パッケージの親パッケージを `...` で参照できる（四重以上のドットも同様）。ドットを使ったインポートを**相対インポート**、そうでないインポートを**絶対インポート**と呼ぶ。

たとえば、`parent` パッケージの例では、`one` サブディレクトリの `__init__.py` の中で、次の import 文

``` python
from . import something
```

これは次の import 文と同等である。

``` python
from parent.one import something
```

また、次の import 文

``` python
from .. import two
```

これは次の import 文と同等である。

``` python
from parent import two
```

相対インポートはパッケージの階層構造に基づくので、パッケージではない（つまり `__path__` 属性を持たない）モジュールにおいては実行できない。たとえば、`python ./a.py` と実行する場合、`a.py` のコードは `__main__` モジュールとして実行されるが、この `__main__` はパッケージではないので、`from . import b` のようなコードはエラーとなる。

なお、`from <MODULE> import *` 文と、`__init__.py` の中で設定される `__all__` 変数について、次の公式チュートリアルを参照すること。

  * パッケージから * を import する: https://docs.python.org/ja/3/tutorial/modules.html#importing-from-a-package

PEP 8 は、`from <MODULE> import *` 文の使用を非推奨とする。実際これを使うと、どのクラスや関数がモジュールからインポートされたものかわかりにくく、またエディタの入力支援が受けられなくなる。

### 動的インポート ###

import 文ではモジュールを識別子で指定するほかないが、インポートするモジュールを識別子ではなく文字列で指定することは**動的インポート**（dynamic imports）と呼ばれる。動的インポートを使うと、たとえばプログラムの実行時に設定ファイルなどに基づいて選択的にモジュールをインポートするということができる。

動的インポートを行うには、標準ライブラリの `importlib` パッケージが提供する `import_module()` 関数を使用する。

``` python
importlib.import_module(name, package=None)
```

この関数は、動的インポートを行い、インポートしたモジュールを返す。`name` 引数は、インポートするモジュールを絶対または相対表現（たとえば `pkg.mod` または `..mod`）の文字列で指定する。`name` が相対表現で与えられたら、`package` 引数を、パッケージ名を解決するためのアンカーとなるパッケージの名前に設定する必要がある（たとえば `import_module('..mod', 'pkg.subpkg')` は `pkg.mod` をインポートする）。

In [ ]:
import importlib

m = "sys"
module = importlib.import_module(m)
module.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

### モジュールの再読み込み ###

インポートシステムには「**再インポートはモジュールを読み込まない**」という特徴がある。これは、複数のモジュールやパッケージの間に依存関係の循環がある場合にモジュールの読み込みが終わらないという事態を避けることが目的である。

例えばトップレベルのコードが `A` モジュールをインポートしていて、`A` モジュールは `B` モジュールを、`B` モジュールは `A` モジュールをインポートしている場合、もし上記の特徴がなければトップレベルのコードを実行すると `A` モジュールの読み込みと `B` モジュールの読み込みが終わらないことになってしまう。

一方、モジュールの再読み込みをしないという特徴により、プログラムの実行中に一度モジュールを読み込むと、モジュールが書き換えられても、それによる変更がプログラムの動作に反映されないことになる。モジュールを開発している場合には、この仕様は不便である。そこで、モジュールの再読み込みを行う関数が用意されている。

``` python
importlib.reload(module)
```

この関数は指定したモジュールを再読み込みする。モジュールを再読み込みすることによって、モジュールに加えた変更をプログラムを再起動せずに利用することができる。

モジュールがインポート済みであるか否かは、組み込み関数 `globals()` または `locals()` が返す辞書のキーを調べるとわかる。たとえば、`sys` モジュールがインポート済みであるなら再読み込みを行いたい場合は、次のように書く:

``` python
if "sys" in locals():
    import importlib
    importlib.reload(sys)  # noqa: F821
```

パッケージング
--------------

Python に関してインストール可能なソフトウェアが「パッケージ」の名で呼ばれるが、これはモジュールの一種としての「パッケージ」とは全く異なる概念である。2 つを区別するため、それぞれ「配布パッケージ」「インポートパッケージ」と呼ぶことがある。

ソースコードから配布パッケージを作成する作業を**パッケージング**（packaging）という。パッケージングにより配布物を作成することで、再利用が簡単になったり、PyPI に登録して公開できるようになる。

パッケージングに必要な作業の流れは以下の通り。

  1. ソースコードツリーを作成する。
  2. `pyproject.toml` ファイルを準備する。
  3. `build` コマンドによって配布物を作成する。

### src-layout と flat-layout ###

ソースコードツリーを決める際には、配布パッケージとインポートパッケージの違いに注意する必要がある。

配布パッケージは Python 言語の外にあるので、その名前は Python 識別子や PEP 8 による制約を受けない。一方、pip の開発元である Python Packaging Authority（PyPA）が配布パッケージのメタデータを標準化している（[PyPA 仕様](https://packaging.python.org/ja/latest/specifications/)）。名前については以下のように定めている。

  1. 名前に使用できる文字は、ASCII 英数字・ピリオド `'.'`・アンダースコア `'_'`・ハイフン `'-'` だけである。
  2. 名前の先頭と末尾は ASCII 英数字でなければならない。

多くの場合、ソースコードツリーのルート（プロジェクトルート）のディレクトリ名を配布パッケージの名前と同じにする。

一方、インポートパッケージは Python 言語におけるモジュールオブジェクトであるから、その名前は Python 識別子や PEP 8 による制約を受ける。PyPA 仕様では許容された先頭の数字とハイフンは使えない。ソースコードを置くディレクトリ名がインポートパッケージの名前となるのでディレクトリ名に注意する。

配布パッケージの名前とインポートパッケージの名前を意図的に異なるものとすることがある。典型的には、開発が終了したプロジェクトをフォークして新しい名前で配布パッケージを公開する場合に、既存コードとの互換性を保持するためにインポートパッケージの名前を旧プロジェクトのままとする事例である。サードパーティ製の画像処理ライブラリである `Pillow` がその例である（`PIL` ライブラリのフォークとして始まった）。

ソースコードツリーの構成には 2 種類ある。ソースコードを置くディレクトリをプロジェクトルートの直下に置くディレクトリ配置を **flat-layout** という。これに対して、ソースコードを置くディレクトリをサブディレクトリに置くディレクトリ配置を **src-layout** という。このサブディレクトリは典型的には `src/` と命名されるので、src-layout と呼ばれる。

src-layout の例:

``` text
my-project  ← カレントディレクトリ
├── README.md
├── pyproject.toml
├── src/
│     └── my_package/
│            ├── __init__.py
│            └── module.py
└── tools/
       ├── generate_awesomeness.py
       └── decrease_world_suck.py
```

flat-layout の例:

``` text
my-project  ← カレントディレクトリ
├── README.md
├── pyproject.toml
├── my_package/
│     ├── __init__.py
│     └── module.py
└── tools/
       ├── generate_awesomeness.py
       └── decrease_world_suck.py
```

flat-layout の場合、Python コードに `import my_project` という import 文があると、デフォルトではカレントディレクトリがモジュール検索パスに追加されるから、`my_project` サブディレクトリがパッケージとしてインポートされる。このため、既にソースコードをパッケージとしてインストールしている場合にも、インポートにはインストール先ではなく編集中のサブディレクトリにあるものが使われる。

src-layout の場合、そのようなことはなく、ソースコードを利用するには「ソースコードのインストール」というステップを踏むことが必要となる。src-layout を採用することにより、今まさに開発中のソースコードを使ってしまうという事故を防ぐことができる。src-layout でも、ソースコードを編集可能モードでインストールすれば、ソースコードの変更をただちにプログラムの動作に反映させることができる。

### pyproject.toml の仕様 ###

`pyproject.toml` は、パッケージングに必要な情報や関連ツールの独自情報をプロジェクト単位で設定するために使用される。その仕様は、[PEP 517](https://peps.python.org/pep-0517/)、[PEP 621](https://peps.python.org/pep-0621/)により標準化されている。このファイルの中には 3 個の TOML テーブルを置くことができる。

  * `[build-system]` テーブルは、プロジェクトをビルドするための設定に使われる。
  * `[project]` テーブルは、プロジェクトの基本的なメタデータを指定するために使われる（配布パッケージのメタデータに取り込まれる）。
  * `[tool]` テーブルは、各関連ツールに特化したサブテーブルを保持するために使われる。

`[build-system]` テーブルには、以下のキーが含まれている。

| キー | データ型 | 意味 |
|:---|:---|:---|
| `build-backend` | 文字列 | 使用するビルドバックエンド |
| `requires`| 文字列の配列 | プロジェクトをビルドするのに必要な依存関係のリスト。`requires = ["setuptools >= 61.0"]` のようにバージョンを制限する<br  />ことができる |

`[project]` テーブルは、以下のキーやサブテーブルを含めることが許されている。

| フィールド名 | データ型 | 意味 |
|:---|:---|:---|
| `dynamic` | 文字列の配列 | 動的であるとマークされるフィールド。動的なメタデータには、値が直接書き込まれるのではなく、ビル<br  />ドバックエンドに計算させるための式が書き込まれる。どのような式を書くかは、ビルドバックエンドの<br  />説明書に従うこと |
| `name` | 文字列 | 配布パッケージ（プロジェクト）の名前。このフィールドは動的であるとマークできない |
| `version` | 文字列 | プロジェクトのバージョン。`major.minor` または `major.minor.micro` の形式とすること |
| `description` | 文字列 | プロジェクトの概要説明 |
| `readme` | 文字列またはテーブル | プロジェクトの完全な説明。`README.md` などのファイルのパスでもよい |
| `requires-python` | 文字列 | プロジェクトの Python バージョン要件。値は `">=3.8"` のようなバージョン指定子 |
| `license` | テーブル | ライセンスを指定するために使われるサブテーブル。`file` キーか `text` キーのいずれか 1 つだけ含<br />む。`file` キーにはプロジェクトのライセンスが含まれるファイルへの相対パスを指定できる。`text` <br />キーにはプロジェクトのライセンスである文字列値を指定できる |
| `authors` | インラインテーブルの配列 | プロジェクトの作者を `name` キーと `email` キーを持つインラインテーブルの配列で指定する |
| `maintainers` | インラインテーブルの配列 | プロジェクトのメンテナーを `name` キーと `email` キーを持つインラインテーブルの配列で指定する |
| `keywords` | 文字列の配列 | プロジェクトのキーワード。PyPI 上の検索でキーワードが与えられた時にこのプロジェクトをサジェスト<br />するのを助ける |
| `classifiers` | 文字列の配列 | プロジェクトに合致する PyPI 分類子 <classifier> のリスト |
| `dependencies` | 文字列の配列またはテーブル | プロジェクトの依存関係。`pip install` と同様のバージョン制約を付けることができる |
| `optional-dependencies` | 文字列の配列またはテーブル | プロジェクトに必須ではない依存関係。たとえば、GUI 用の依存関係や開発専用の依存関係など。サブ<br />テーブルとしたときは、その中のキー `key` を指定して `pip install your-project-name[key]` を実<br />行すると、追加の依存関係付きでインストールすることができる |
| `urls` | テーブル | プロジェクトに関連のある URL を保持するサブテーブル。キーの名前はそのまま PyPI 上のプロジェク<br />トページの左サイドバーに表示される項目名に使われる |
| `scripts` | テーブル | プロジェクトがコマンドもインストールされるパッケージである場合にコマンドを生成するために使われ<br />るサブテーブル。たとえば、このテーブルに `spam-cli = "spam:main_cli"` があると<br /> `from spam import main_cli; main_cli()` と同等のことを行うコマンド `spam-cli` が生成される |
| `gui-scripts` | テーブル | プロジェクトが Windows で GUI を提供するパッケージである場合に GUI を起動するコマンドを生成す<br />るために使われるサブテーブル。使い方は `[project.scripts]` サブテーブルと同じ |
| `entry-points` | テーブル | プロジェクトが特定のパッケージのプラグインであることを宣言するためのテーブルを保持するために<br />使われるサブテーブル |

`name` と `version` は、必ず `[project]` テーブルに含まれていなければならない。

バージョン番号の流儀は、Python のバージョン番号を参考にするとよい（[公式 FAQ](https://docs.python.org/ja/3/faq/general.html#how-does-the-python-version-numbering-scheme-work) より）。

>Python のバージョン番号は "A.B.C" または "A.B" が付けられます:
>
>  * A はメジャーバージョン番号です -- 言語に本当に大きな変更があった場合に増やされます。
>  * B はマイナーバージョン番号です -- 驚きの少ない変更があった場合に増やされます。
>  * C はマイクロバージョン番号です -- バグフィックスリリースごとに増やされます。

自作ライブラリをパッケージング可能にするには、 `pyproject.toml` に `[build-system]` テーブルと `[project]` テーブルを置く必要がある。

`[project]` テーブルの `dependencies` フィールドの設定は、`pip install` コマンドがインストールすべき依存パッケージを探し出すためのメタデータに使われる。開発中のプロジェクトで使用した `requirements.txt` と `dependencies` フィールドとで内容に矛盾があると、配布パッケージのインストールが正しく行われない。この 2 つを同期させる必要があり、これを手作業で行うか、[uv](https://docs.astral.sh/uv/) などのプロジェクト管理ツールを使って自動化することになる。

次の TOML 形式は `pyproject.toml` の例である。

``` ini
[build-system]
requires = ["setuptools >= 77.0.3"]
build-backend = "setuptools.build_meta"

[project]
name = "spam-eggs"
version = "2020.0.0"
dependencies = [
  "httpx",
  "gidgethub[httpx]>4.0.0",
  "django>2.1; os_name != 'nt'",
  "django>2.0; os_name == 'nt'",
]
requires-python = ">=3.8"
authors = [
  {name = "Pradyun Gedam", email = "pradyun@example.com"},
  {name = "Tzu-Ping Chung", email = "tzu-ping@example.com"},
  {name = "Another person"},
  {email = "different.person@example.com"},
]
maintainers = [
  {name = "Brett Cannon", email = "brett@example.com"}
]
description = "Lovely Spam! Wonderful Spam!"
readme = "README.rst"
license = {file = "LICENSE.txt"}
keywords = ["egg", "bacon", "sausage", "tomatoes", "Lobster Thermidor"]
classifiers = [
  "Development Status :: 4 - Beta",
  "Programming Language :: Python"
]

dependencies = [
  "httpx",
  "gidgethub[httpx]>4.0.0",
  "django>2.1; os_name != 'nt'",
  "django>2.0; os_name == 'nt'"
]

[project.optional-dependencies]
test = [
  "pytest < 5.0.0",
  "pytest-cov[all]"
]

[project.urls]
Homepage = "https://example.com"
Documentation = "https://readthedocs.org"
Repository = "https://github.com/me/spam.git"
"Bug Tracker" = "https://github.com/me/spam/issues"
Changelog = "https://github.com/me/spam/blob/master/CHANGELOG.md"

[project.scripts]
spam-cli = "spam:main_cli"

[project.gui-scripts]
spam-gui = "spam:main_gui"

[project.entry-points."spam.magical"]
tomatoes = "spam:main_tomatoes"
```

### sdist と wheel ###

パッケージとしてインストール可能な配布物の形式には次の 2 種類がある。

  * **sdist**:  
ソースコードとメタデータを持つテキストファイルをアーカイブしたファイル形式。C などで書かれた拡張がソースコードに含まれていると、インストールにコンパイルが必要となる。
  * **wheel**:  
ソースコードとメタデータを持つテキストファイル、およびコンパイル済みバイナリファイルをアーカイブしたファイル形式。拡張子は `.whl`。Python ソースコードのみのパッケージ（ピュア Python パッケージ）では、多くの場合 1 つの wheel ファイルで済む。コンパイル済みのバイナリ拡張を伴うパッケージでは、そのパッケージがサポートする Python インタープリター・オペレーティングシステム・ CPU アーキテクチャの組み合わせのそれぞれについて wheel ファイルが必要になる。

配布物の作成には [build](https://build.pypa.io/en/stable/) を使用する。`build` は PyPA が開発しているが、公式の Python インストーラーに同梱されていないのでインストールする必要がある。ライセンスは MIT license。インストール方法は次のとおり。

``` shell
pip install build
```

次のコマンドを実行することで 2 つの形式の配布物を作成する:

``` shell
python -m build <source-tree-directory>
```

どちらか一方の形式だけ作成するには、オプションを付ける:

``` shell
python -m build --sdist <source-tree-directory>
python -m build --wheel <source-tree-directory>
```

簡単な例を示す。カレントディレクトリに以下のような src-layout となるソースコードツリーがある:

``` python
.
├── pyproject.toml
└── src/
       └── greeting/
              ├── __init__.py
              ├── __main__.py
              └── say.py
```

`__init__.py` は空ファイルである。他のファイルについては以下の通り。

`__main__.py`:

``` python
from . import say

def main():
    say.say()

if __name__ == "__main__":
    main()
```

`say.py`:

``` python
def say():
    print("Hello")
```

`pyproject.toml`:

``` ini
[build-system]
requires = ["setuptools >= 77.0.3"]
build-backend = "setuptools.build_meta"

[project]
name = "greeting"
version = "1.0.0"

[project.scripts]
greet = "greeting.__main__:main"
```

今回は PyPI に登録しないので、`pyproject.toml` は最小限の設定だけとした。ただし、今回は実行形式も作成したいので `[project.scripts]` サブテーブルを記述している。

カレントディレクトリに仮想環境を作成し、有効化しておく。src-layout なので、`python -m greeting` を実行しても `No module named greeting` というエラーが出る。

`build` をインストールしてから、次のコマンドを実行する:

``` shell
python -m build .
```

すると、`build` はビルドバックエンドを認識して `setuptools` パッケージをインストールし、`dist` サブディレクトリを作成して、そこにパッケージングの結果を保存する。

``` python
.
├── pyproject.toml
├── dist/
│     ├── greeting-1.0.0.tar.gz
│     └── greeting-1.0.0-py3-none-any.whl
└── src/
       └── greeting/
              ├── __init__.py
              ├── __main__.py
              └── say.py
```

`dist/greeting-1.0.0.tar.gz` が sdist 形式であり、`dist/greeting-1.0.0-py3-none-any.whl` が wheel 形式である。

現在の仮想環境に自作パッケージをインストールしてみる。どちらの形式でもよいが、次のコマンドでは wheel 形式の配布物を指定している。

``` shell
pip install dist/greeting-1.0.0-py3-none-any.whl
```

インストールに成功したら、`python -m greeting` を実行すると `Hello` という表示が出る。

仮想環境の `bin` サブディレクトリ（Windows では `Scripts` サブディレクトリ）に実行形式も作成されているので `greet` コマンドを実行しても `Hello` という表示が出る。

パス設定ファイル
----------------

ユーザーによってインストールされたパッケージのファイルやメタデータが保存されるディレクトリのことを **site-packages ディレクトリ**と呼ぶ。

Python インタープリターを起動すると標準ライブラリの `site` モジュールが自動的にインポートされ、そのインポート時に呼び出される `site.main()` 関数によって site-packages ディレクトリが `sys.path` に追加される。Python インタープリターの `-S` オプションで `site` 自動インポートを禁止できる。

モジュール変数 `site.USER_SITE` で site-packages ディレクトリを確認できる。また、モジュール変数 `site.ENABLE_USER_SITE` が `True` の場合、 site-packages ディレクトリが `sys.path` に追加されている。

In [ ]:
import site
print(f"{site.USER_SITE=}")
print(f"{site.ENABLE_USER_SITE=}")

site.USER_SITE='/root/.local/lib/python3.10/site-packages'
site.ENABLE_USER_SITE=True


Python 環境の site-packages ディレクトリにある拡張子 `.pth` のファイルを**パス設定ファイル**と呼ぶ。`site.main()` 関数は自動的にパス設定ファイルを読み込み、そこに書かれたディレクトリのパスをモジュール探索パス（`sys.path`）に追加する。

開発中のパッケージのソースコードを編集可能モード（`pip install` の `-e` オプション）でインストールする場合、 site-packages ディレクトリにはパッケージそのものではなく、パス設定ファイルが追加され、パス設定ファイルの中に開発中のパッケージのパスが書き込まれる。`site.main()` 関数がこれを読み込んでモジュール探索パスに追加するので、開発中のソースコードがインポートされる仕組みになっている。つまり、編集可能モードでのインストールは、モジュール探索パスの追加でしかない。

先の自作 `greeting` パッケージを使って編集可能モードを試してみる。パッケージをインストールした仮想環境を一旦削除し、新たに仮想環境を作り直して有効化しておく。src-layout なので、`python -m greeting` を実行しても `No module named greeting` というエラーが出る。

次のコマンドでソースコードを編集可能モードでインストールする。

``` shell
pip install -e .
```

`pip install` コマンドは、ソースコードをインストールする場合は `pyproject.toml` を使用する（`pyproject.toml` がないとエラーとなる）。インストールに成功したら、`python -m greeting` を実行すると `Hello` が表示される。仮想環境の `bin` サブディレクトリ（Windows では `Scripts` サブディレクトリ）に実行形式も作成されているので `greet` コマンドを実行しても `Hello` という表示が出る。

編集可能モードの効果を確かめるために、`say.py` の `say()` 関数を次のように書き換える:

``` python
def say():
    print("Good morning")
```

すると、`python -m greeting` を実行しても `greet` コマンドを実行しても `Good morning` が表示される。再インストールしなくてもソースコードの変更が反映された。これが編集可能モードの効果である。

仮想環境の site-packages ディレクトリの中を調べてみると、以下のディレクトリとファイルが作成されている:

  * `greeting-1.0.0.dist-info` ディレクトリ
  * `__editable__.greeting-1.0.0.pth`

`greeting-1.0.0.dist-info` ディレクトリはパッケージのメタデータを格納しているだけで、ソースコードは含まれていない。`__editable__.greeting-1.0.0.pth` はパス設定ファイルであり、中身はプロジェクトの `src` サブディレクトリのパスが書かれている。`site.main()` 関数はこのパス設定ファイルを読み込んで `src` サブディレクトリをモジュール探索パスに追加する。こうして `src` サブディレクトリ直下の `greeting` ディレクトリがパッケージとしてインポートされるので、再インストールなしにソースコードの変更が反映されるわけである。